In [1]:
import pandas as pd

In [2]:
# Archivos Excel para Cargar en las tablas de la Base de Datos posgres

datos_colon = pd.read_excel("ArchivosExcel/EducacionColon.xlsx", sheet_name="TotalEducacionColon")

datos_ischilin = pd.read_excel("ArchivosExcel/EducacionIschilin.xlsx", sheet_name="TotalEducacionIschilin")

datos_totoral = pd.read_excel("ArchivosExcel/EducacionTotoral.xlsx", sheet_name="TotalEducacionTotoral")

datos_tulumba = pd.read_excel("ArchivosExcel/EducacionTulumba.xlsx", sheet_name="TotalEducacionTulumba")

In [6]:
import conect
import logging


def cargar_datos_en_base_de_datos():
    try:
        # Conectar a la base de datos
        conexion = conect.connect_to_db()
        
        if conexion is None:
            raise Exception("No se pudo conectar a la base de datos")
        
        # Archivos Excel
        archivos_excel = [
            ("ArchivosExcel/EducacionColon.xlsx", "TotalEducacionColon", "datos_educ_colon"),
            ("ArchivosExcel/EducacionIschilin.xlsx", "TotalEducacionIschilin", "datos_educ_ischilin"),
            ("ArchivosExcel/EducacionTotoral.xlsx", "TotalEducacionTotoral", "datos_educ_totoral"),
            ("ArchivosExcel/EducacionTulumba.xlsx", "TotalEducacionTulumba", "datos_educ_tulumba")
        ]
        
        # Cursor
        cur = conexion.cursor()
        
        # Iterar sobre los archivos Excel
        for archivo, sheet_name, tabla in archivos_excel:
            try:
                # Leer archivo Excel
                datos = pd.read_excel(archivo, sheet_name=sheet_name)
                
                # Crear lista de tuplas
                data = [tuple(row) for row in datos.values]
                
                # Insertar datos en tabla
                cur.executemany(f"""
                    INSERT INTO {tabla} VALUES ({','.join(['%s'] * len(datos.columns))})
                """, data)
            
            except pd.errors.EmptyDataError:
                logging.warning(f"Archivo {archivo} está vacío")
            
            except pd.errors.ParserError as e:
                logging.error(f"Error al parsear archivo {archivo}: {e}")
            
            except Exception as e:
                logging.error(f"Error al cargar datos en tabla {tabla}: {e}")
        
        # Commit de los cambios
        conexion.commit()
        
        # Cerrar cursor y conexión
        cur.close()
        conexion.close()
        
        print("Datos cargados con éxito")
    
    except Exception as e:
        logging.error(f"Error general: {e}")
        print("No se pudo cargar los datos")


cargar_datos_en_base_de_datos()

Conexión Exitosa
Datos cargados con éxito
